# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
This is just a baseline model for evidence retrieval. You may have no need to run it as it is just a baseline that we used.

We uploaded this just because we still spent a lot of time on it actually so we hope this can be seen as our efforts for this project, too.

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

1.Clone the repository to get the data

In [1]:
import os

# the repository link:
repository_url = 'https://github.com/drcarenhan/COMP90042_2024.git'

# clone the repository
os.system(f'git clone {repository_url}')


32768

We still need to download the evidence data from the google drive as it is not in the repository.

In [2]:
save_path = '/content/COMP90042_2024/data'
os.makedirs(save_path, exist_ok=True)

output_file_path = os.path.join(save_path, 'evidence.json')

!gdown --id '1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6' -O {output_file_path}

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6
From (redirected): https://drive.google.com/uc?id=1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6&confirm=t&uuid=21dad9c4-9125-438f-b54d-23b0edfce037
To: /content/COMP90042_2024/data/evidence.json
100% 174M/174M [00:01<00:00, 87.8MB/s]


Read all the related data

In [3]:
import json

with open(save_path+'/train-claims.json', 'r') as file:
    train_claims_data = json.load(file)

with open(save_path+'/evidence.json', 'r') as file:
    evidence_data = json.load(file)

with open(save_path+'/dev-claims.json', 'r') as file:
    dev_claims_data = json.load(file)

with open(save_path+'/dev-claims-baseline.json', 'r') as file:
    dev_claims_baseline_data = json.load(file)

with open(save_path+'/test-claims-unlabelled.json', 'r') as file:
    test_claims_unlabelled_data = json.load(file)

Making the traning set pairs (claim,evidence,label)

In [4]:
# Extract all evidence IDs used in train_claims_data
train_evidence_ids = set()
for claim in train_claims_data.values():
    train_evidence_ids.update(claim['evidences'])

# Create a subset of evidence_data containing only those evidences referenced in train_claims_data
train_evidence_subset = {eid: evidence_data[eid] for eid in train_evidence_ids if eid in evidence_data}

# Extract all evidence IDs used in train_claims_data
dev_evidence_ids = set()
for claim in dev_claims_data.values():
    dev_evidence_ids.update(claim['evidences'])

# Create a subset of evidence_data containing only those evidences referenced in train_claims_data
dev_evidence_subset = {eid: evidence_data[eid] for eid in dev_evidence_ids if eid in evidence_data}


### Preprocessing

prepare the training dataset that claim and evidence are stored separately

In [5]:
import numpy as np

positive_train_pairs =  []
for claim_id, claim_infor in train_claims_data.items():
    for evidence_id in claim_infor['evidences']:
        positive_train_pairs.append((claim_id, evidence_id))

positive_train_text_pairs = []
for claim_id, evidence_id in positive_train_pairs:
    positive_train_text_pairs.append((train_claims_data[claim_id]['claim_text'], evidence_data[evidence_id]))

positive_dev_pairs =  []
for claim_id, claim_infor in dev_claims_data.items():
    for evidence_id in claim_infor['evidences']:
        positive_dev_pairs.append((claim_id, evidence_id))

positive_dev_text_pairs = []
for claim_id, evidence_id in positive_dev_pairs:
    positive_dev_text_pairs.append((dev_claims_data[claim_id]['claim_text'], evidence_data[evidence_id]))
# Get the question and answer list
train_claim_list = [pair[0] for pair in positive_train_text_pairs]
train_evidence_list = [pair[1] for pair in positive_train_text_pairs]

dev_claim_list = [pair[0] for pair in positive_dev_text_pairs]
dev_evidence_list = [pair[1] for pair in positive_dev_text_pairs]


In [6]:
n_data = len(positive_train_pairs)
n_data

4122

In [7]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from nltk.stem import *
from nltk.stem import WordNetLemmatizer

# Have a look at some contractions of words! Contractions include punctuation - how would you handle them? There are many edge cases. i.e. can't, she'll
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def pre_process(sent_list):
    output = []
    for sent in tqdm(sent_list):
        sent = sent.lower()  # case-folding
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word)  # dealing with contractions
        sent = re.sub(r'[^\w\s]', '', sent)  # removing punctuation
        tokens = word_tokenize(sent)  # tokenization
        tokens = [word for word in tokens if word not in stop_words]
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        tokens = [stemmer.stem(word) for word in tokens]
        if len(tokens) > 0:
          output.append(["<BOS>"] + tokens + ["<EOS>"])
        else:
          output.append(tokens)
    return output

# Preprocessing the data using the function defined above
input_token_list_claim = pre_process(train_claim_list)  # -> input to encoder
input_token_list_evidence = pre_process(train_evidence_list)
dev_input_token_list_claim = pre_process(dev_claim_list)  # -> input to encoder
dev_input_token_list_evidence = pre_process(dev_evidence_list)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|██████████| 491/491 [00:00<00:00, 911.72it/s]


In [8]:
# Here we simply use the maximum sentence length
MAX_LENGTH = max([len(s) for s in input_token_list_claim]+[len(s) for s in input_token_list_evidence])
MAX_LENGTH = 96

In [9]:
all_lengths = [len(s) for s in input_token_list_claim] + [len(s) for s in input_token_list_evidence]

# calculate the avg length for texts
average_length = sum(all_lengths) / len(all_lengths)

In [10]:
# set up a vocab to index dictionary
word_to_ix = {"<BOS>": 0, "<EOS>":1, "<UNK>":2}
for sentence in input_token_list_claim + input_token_list_evidence:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

In [11]:
# construct token index lists for input
# i.e., to convert each sequence into corresponding index based on the index dictionary we created in the previous section
# e.g., ['are', 'your', 'systems', 'functioning'] -> [42, 6, 2576, 2577]
unk_index = word_to_ix.get("<UNK>", 2)
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([float(to_ix.get(w, unk_index)) for w in sent])
    return input_index_list

input_index_1 = to_index(input_token_list_claim, word_to_ix)
input_index_2 = to_index(input_token_list_evidence, word_to_ix)
dev_input_index_1 = to_index(dev_input_token_list_claim, word_to_ix)
dev_input_index_2 = to_index(dev_input_token_list_evidence, word_to_ix)
dev_train_input_index_1 = input_index_1 + dev_input_index_1
dev_train_input_index_2 = input_index_2 + dev_input_index_2

### dataloader implementation

In [12]:
import torch

MAX_LENGTH = 96
max_len = MAX_LENGTH
padding_index = word_to_ix["<EOS>"]
def seq_collate_batch(batch):
    claim_list,evidence_list = [],[]
    for  claim_text,evidence_text in batch:
        claim_list.append(claim_text)
        evidence_list.append(evidence_text)

    # Pad or truncate each sequence
    padded_sequences = []
    for seq in claim_list:
        # Truncate if longer than max_len
        padded_seq = seq[:max_len]
         # Pad if shorter
        padded_seq += [padding_index] * (max_len - len(padded_seq))
        padded_sequences.append(torch.LongTensor(padded_seq))
    claim_list = torch.stack(padded_sequences)
    # Pad or truncate each sequence
    padded_sequences = []
    for seq in evidence_list:
        # Truncate if longer than max_len
        padded_seq = seq[:max_len]
         # Pad if shorter
        padded_seq += [padding_index] * (max_len - len(padded_seq))
        padded_sequences.append(torch.LongTensor(padded_seq))
    evidence_list = torch.stack(padded_sequences)
    # Stack all sequences into a single tensor
    return claim_list.to(device), evidence_list.to(device)


In [13]:
train_dataloader = torch.utils.data.DataLoader(list(zip(input_index_1, input_index_2)), batch_size=8, shuffle=True, collate_fn=seq_collate_batch)
dev_dataloader = torch.utils.data.DataLoader(list(zip(dev_input_index_1, dev_input_index_2)), batch_size=4, shuffle=True, collate_fn=seq_collate_batch)

### models

In [14]:
import torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Bi-LSTM model implementation

In [15]:
import torch.nn as nn
import torch

class EncoderBiLSTM(nn.Module):
    def __init__(self, hidden_size, input_size,embedding):
        super(EncoderBiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True,batch_first=True)

    def forward(self, x, hc):
        embedded = self.embedding(x)
        output, (h_n, c_n) = self.lstm(embedded,hc)

        # h_n shape (num_layers * num_directions, hidden_size)
        # For Bi-LSTM, num_directions = 2
        forward_hidden = h_n[-2, :]
        backward_hidden = h_n[-1, :]
        forward_cell = c_n[-2, :]
        backward_cell = c_n[-1, :]
        embeddings = torch.cat((forward_hidden, backward_hidden), dim=1)
        return output, embeddings, (forward_cell,backward_cell), (h_n, c_n)

    def initHidden(self,batch_size):
        num_directions = 2
        return (torch.zeros(num_directions,batch_size, self.hidden_size, device=device),
                torch.zeros(num_directions,batch_size, self.hidden_size, device=device))


In [16]:
def train(input_tensor_1, input_tensor_2, encoder_1, encoder_2, encoder_optimizer_1, encoder_optimizer_2, max_length=MAX_LENGTH):

    loss = 0
    encoder_optimizer_1.zero_grad()
    encoder_optimizer_2.zero_grad()

    # get the embeddings for claims and evidences inside a batch (shape:  batch_size * hidden_size)
    encoder_output_1, claim_embeddings, _, _ = encoder_1(input_tensor_1, encoder_1.initHidden(input_tensor_1.shape[0]))
    encoder_output_2, evidence_embeddings, _, _ = encoder_2(input_tensor_2, encoder_2.initHidden(input_tensor_2.shape[0]))
    #print(claim_embeddings.shape)
    claim_embeddings = claim_embeddings.reshape(-1, 2 * hidden_size)
    evidence_embeddings = evidence_embeddings.reshape(-1, 2 * hidden_size)
    #caculate a similarity matrix for dot product of claim and evidence embeddings inside a batch
    similarity_matrix =  torch.matmul(claim_embeddings, evidence_embeddings.T)

    claim_to_evidences = {}

    for i,input_1 in enumerate(input_tensor_1):
        for j, input_2 in enumerate(input_tensor_1):
            if torch.equal(input_1,input_2):
               claim_to_evidences.update({i:j})

    if len(claim_to_evidences) > 32:
        print(claim_to_evidences)
    batch_size = input_tensor_1.shape[0]
    labels = torch.zeros((batch_size, batch_size)).to(similarity_matrix.device)
    for claim_idx, evidence_idx in claim_to_evidences.items():
        labels[claim_idx, evidence_idx] = 1

    log_probs = F.log_softmax(similarity_matrix, dim=1)

    loss = -torch.sum(log_probs * labels) / torch.sum(labels)

    loss.backward()
    encoder_optimizer_1.step()
    encoder_optimizer_2.step()

    return loss.item()

In [17]:
import time
import math

# Helper functions for training
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [18]:
import random
from tqdm import tqdm
def trainIters(encoder_1, encoder_2,dataloader, epochs,learning_rate=1e-4):
    start = time.time()

    encoder_optimizer_1 = optim.Adam(encoder_1.parameters(), lr=learning_rate)
    encoder_optimizer_2 = optim.Adam(encoder_2.parameters(), lr=learning_rate)

    for iter in range(epochs):
        size = len(dataloader.dataset)
        for batch, (input_tensor_1, input_tensor_2) in enumerate(dataloader):

            loss = train(input_tensor_1, input_tensor_2, encoder_1, encoder_2, encoder_optimizer_1, encoder_optimizer_2)
        print("epoch:" + str(iter) + " loss: " + str(loss))


In [19]:
hidden_size = 200
embedding = nn.Embedding(len(word_to_ix), hidden_size,padding_idx = padding_index)
encoder1 = EncoderBiLSTM(hidden_size, hidden_size, embedding).to(device)
encoder2 = EncoderBiLSTM(hidden_size, hidden_size, embedding).to(device)
trainIters(encoder1, encoder2, train_dataloader,epochs = 25)

epoch:0 loss: 0.6487799286842346
epoch:1 loss: 0.8724571466445923
epoch:2 loss: 0.49407118558883667
epoch:3 loss: 0.10495072603225708
epoch:4 loss: 0.052710555493831635
epoch:5 loss: 0.01569930650293827
epoch:6 loss: 0.00021542527247220278
epoch:7 loss: 0.058339111506938934
epoch:8 loss: 0.002314769197255373
epoch:9 loss: 0.0003003384335897863
epoch:10 loss: 0.00010030500561697409
epoch:11 loss: 5.483381755766459e-05
epoch:12 loss: 0.002915847348049283
epoch:13 loss: 0.00017878196376841515
epoch:14 loss: 2.7894195227418095e-05
epoch:15 loss: 7.771843229420483e-05
epoch:16 loss: 0.005906157195568085
epoch:17 loss: 0.00020160229178145528
epoch:18 loss: 5.9604641222676946e-08
epoch:19 loss: 5.125981715536909e-06
epoch:20 loss: 6.746986764483154e-05
epoch:21 loss: 0.0015716117341071367
epoch:22 loss: 1.4364518392540049e-05
epoch:23 loss: 2.145764938177308e-06
epoch:24 loss: 6.616093742195517e-06


# 3.Testing and Evaluation

## predict on the test set

In [31]:
#read the test dataset json files
with open('/content/COMP90042_2024/data/test-claims-unlabelled.json', 'r') as f:
  test_claims = json.load(f)

test_evidence_ids = set()
# Initialize an empty list to store the paired data

### pre-compute all evidence embeddings in advance and index them using FAISS

preprocess all evidence text

In [20]:
from tqdm import tqdm
all_evidence_text_list = []
all_evidence_id_list = []
processed_all_evidence_text_list = []
for evidence_id, evidence_text in tqdm(evidence_data.items()):
    all_evidence_text_list.append(evidence_text)
    all_evidence_id_list.append(evidence_id)

processed_all_evidence_text_list = pre_process(all_evidence_text_list)
# filter out the corresponding evidence id which has empty evidence text
processed_all_evidence_id_list = [
    evidence_id for evidence_id, evidence_text in tqdm(zip(all_evidence_id_list, processed_all_evidence_text_list))
    if evidence_text and not (evidence_text == "<BOS><EOS>")
]

processed_all_evidence = dict(zip(processed_all_evidence_id_list, [text for text in processed_all_evidence_text_list if text]))

print(len(processed_all_evidence_id_list))
print(len(all_evidence_text_list))
print(len(processed_all_evidence))

100%|██████████| 1208827/1208827 [13:00<00:00, 1547.99it/s]
1208827it [00:00, 2339573.47it/s]


1208577
1208827
1208577


In [62]:
json.dump(processed_all_evidence, open('/content/processed_all_evidence.json', 'w'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
#load the processed_all_evidence.json
import json
with open('/content/drive/MyDrive/COMP90042_PROJECT/processed_all_evidence.json', 'r') as f:
  processed_all_evidence = json.load(f)

In [21]:
!pip install faiss-cpu

Compute the evidence embeddings using the trained evidence encoder (encoder2)

In [22]:
processed_all_evidence_list = []
processed_all_evidence_text_list = []
for evidence_id, evidence_text in tqdm(processed_all_evidence.items()):
    processed_all_evidence_text_list.append(evidence_text)
all_evidence_input_index_list = to_index(processed_all_evidence_text_list,word_to_ix)

evidence_ids = list(processed_all_evidence.keys())
processed_all_evidence_list = list(zip(evidence_ids, all_evidence_input_index_list))

100%|██████████| 1208577/1208577 [00:00<00:00, 1336766.27it/s]


In [23]:
max_len = MAX_LENGTH
padding_index = word_to_ix["<EOS>"]
def test_collate_batch(batch):
    text_list,id_list = [],[]
    for  _id, _text in batch:
        text_list.append(_text)
        id_list.append(_id)
    # Pad or truncate each sequence
    padded_sequences = []
    for seq in text_list:
        # Truncate if longer than max_len
        padded_seq = seq[:max_len]
         # Pad if shorter
        padded_seq += [padding_index] * (max_len - len(padded_seq))
        #print(len(padded_seq))
        padded_sequences.append(torch.LongTensor(padded_seq))
    text_list = torch.stack(padded_sequences)

    return id_list, text_list.to(device)

evidence_dataloader = torch.utils.data.DataLoader(processed_all_evidence_list, batch_size=512, shuffle=True, collate_fn=test_collate_batch)

In [24]:
evidence_embeddings = {}
with torch.no_grad():
    for batch, (evidence_id_batch, evidence_input_index_tensor_batch) in enumerate(tqdm(evidence_dataloader)):
        encoder_output, encoder_embedding, _, last_encoder_hidden_cell = encoder2(evidence_input_index_tensor_batch, encoder2.initHidden(evidence_input_index_tensor_batch.shape[0]))

        for _id, _hidden in zip(evidence_id_batch, encoder_embedding):
            evidence_embeddings.update({_id:_hidden})


100%|██████████| 2361/2361 [01:19<00:00, 29.86it/s]


In [25]:
evidence_embeddings_list = []
evidence_embeddings_list = [evidence_embeddings[evidence_id].cpu() for evidence_id in evidence_ids]

In [26]:
import faiss
evidence_embeddings_list = np.array(evidence_embeddings_list)
vector_dimension = evidence_embeddings_list.shape[1]
index = faiss.IndexFlatIP(vector_dimension)
faiss.normalize_L2(evidence_embeddings_list)
index.add(evidence_embeddings_list)

### compute the claim embeddings and evaluate the model on dev dataset

In [27]:
dev_claims_text_list = [claim_infor['claim_text'] for claim_id,claim_infor in dev_claims_data.items()]
dev_claims_input_index_list = to_index(pre_process(dev_claims_text_list),word_to_ix)
dev_claims_id_list = list(dev_claims_data.keys())
dev_claims_input_list = list(zip(dev_claims_id_list, dev_claims_input_index_list))
dev_claim_dataloader = torch.utils.data.DataLoader(dev_claims_input_list, batch_size=64, shuffle=True, collate_fn=test_collate_batch)

100%|██████████| 154/154 [00:00<00:00, 2056.07it/s]


In [28]:
dev_claim_retrieved_evidence = {}
with torch.no_grad():
    for batch, (claim_id_batch, claim_input_index_tensor_batch) in enumerate(tqdm(dev_claim_dataloader)):
        encoder_output, encoder_embedding, _, last_encoder_hidden_cell = encoder1(claim_input_index_tensor_batch, encoder1.initHidden(claim_input_index_tensor_batch.shape[0]))
        k = 3  # Number of nearest neighbors to retrieve
        encoder_embedding = encoder_embedding.cpu().numpy()
        faiss.normalize_L2(encoder_embedding)
        distances, indices = index.search(encoder_embedding, k)
        for i, claim_id in enumerate(claim_id_batch):
            dev_claim_retrieved_evidence.update({claim_id: [evidence_ids[idx] for idx in indices[i]] })

100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


In [29]:
dev_claim_ids = set([claim_id for claim_id,claim_infor in dev_claims_data.items()])

true_positive = 0
false_positive = 0
false_negative = 0
for claim_id in dev_claim_ids:
    gold_evidence = set([evidence_id for evidence_id in dev_claims_data[claim_id]['evidences']])
    predicted_evidence = set([evidence_id for evidence_id in dev_claim_retrieved_evidence[claim_id]])
    true_positive += len(gold_evidence & predicted_evidence)
    false_positive += len(predicted_evidence - gold_evidence)
    false_negative += len(gold_evidence - predicted_evidence)

accuracy_evidence_retrieval = true_positive / (true_positive + false_positive + false_negative)
recall_evidence_retrieval = true_positive / (true_positive + false_negative)
f1_evidence_retrieval = 2 * true_positive / (2 * true_positive + false_positive + false_negative)
print("Evidence Retrieval")
print("Accuracy:", accuracy_evidence_retrieval)
print("Recall:", recall_evidence_retrieval)
print("F1 Score:", f1_evidence_retrieval)

Evidence Retrieval
Accuracy: 0.005274261603375527
Recall: 0.010183299389002037
F1 Score: 0.01049317943336831


### test set inference

In [32]:
test_claims_text_list = [claim_infor["claim_text"] for claim_id,claim_infor in test_claims.items()]
test_claims_input_index_list = to_index(pre_process(test_claims_text_list),word_to_ix)
test_claims_id_list = list(test_claims.keys())
test_claims_input_list = list(zip(test_claims_id_list, test_claims_input_index_list))
test_claim_dataloader = torch.utils.data.DataLoader(test_claims_input_list, batch_size=64, shuffle=True, collate_fn=test_collate_batch)

100%|██████████| 153/153 [00:00<00:00, 1153.87it/s]


In [33]:
test_claim_retrieved_evidence = {}
with torch.no_grad():
    for batch, (claim_id_batch, claim_input_index_tensor_batch) in enumerate(tqdm(test_claim_dataloader)):
        encoder_output, encoder_embedding, _, last_encoder_hidden_cell = encoder1(claim_input_index_tensor_batch, encoder1.initHidden(claim_input_index_tensor_batch.shape[0]))
        k = 3  # Number of nearest neighbors to retrieve
        distances, indices = index.search(encoder_embedding.cpu().numpy(), k)
        for i, claim_id in enumerate(claim_id_batch):
            test_claim_retrieved_evidence.update({claim_id: [evidence_ids[idx] for idx in indices[i]] })

100%|██████████| 3/3 [00:13<00:00,  4.64s/it]


In [34]:
for claim_id, claim_data in tqdm(test_claims.items(), desc="Processing Claims"):
    test_claims[claim_id]['claim_label'] = "SUPPORTS"
    test_claims[claim_id]['evidences'] = []
    for evidence_id in test_claim_retrieved_evidence[claim_id]:
        test_claims[claim_id]['evidences'].append(evidence_id)

json.dump(test_claims, open('/content/COMP90042_2024/data/test-output.json', 'w'))

Processing Claims: 100%|██████████| 153/153 [00:00<00:00, 198554.61it/s]


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*